In [7]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

In [4]:
base=pd.read_csv('PETR4.SA.csv')
base=base.dropna()
base_treinamento=base.iloc[:, 1:2].values

In [8]:
normalizador=MinMaxScaler(feature_range=(0,1))
base_treinamento_normalizada = normalizador.fit_transform(base_treinamento)